### Demo #3 Using LLM to assist with submission grading tasks
This demo shows how you can use LLMs to help perform a grading task with a rubic. The example uses an established SIB data management rubric to assess SIB submissions involving data mangement. 

We will setup individual agents to evaulate each criteria and a final grader agent to consolidate all the results from each agent to make a recommendation on the submission.

from langchain.agents.agent_types import AgentType
from langchain_openai import AzureChatOpenAI

from dotenv import load_dotenv

import os